#  **🔧Functions:** 
LangChain provides powerful helper methods to extend the functionality of **Runnables** in LCEL pipelines. Below is a breakdown of the most useful ones with examples:

## **🔹 What is `.assign()` in LangChain?**
✅ `.assign()` is a **helper method** that allows you to **add new computed fields** to an existing dictionary **without modifying the original input**.  
✅ It is often used with **`RunnablePassthrough`** to **pass through existing data** while adding additional fields.

---

## **🔹 How `.assign()` Works**
### **General Syntax**
```python
RunnablePassthrough().assign(new_key=lambda inputs: some_computation(inputs))
```

### `.assign()` expects a dictionary as an input

### **👀 Example 1: Adding a Word Count to Text**

In [5]:
from langchain_core.runnables import RunnablePassthrough

pipeline = RunnablePassthrough()

# Invoke the pipeline
output = pipeline.invoke({"text": "LangChain is awesome!"})
print(output)


############################################################


# Create a passthrough pipeline and assign a new field
pipeline = RunnablePassthrough().assign(
    word_count=lambda inputs: len(inputs["text"].split())
)

# Invoke the pipeline
output = pipeline.invoke({"text": "LangChain is awesome!"})
print(output)

{'text': 'LangChain is awesome!'}
{'text': 'LangChain is awesome!', 'word_count': 3}


### **👀 Example 2: Combining `.assign()` with LLMs**
**Scenario:**
- Run an LLM to generate text.
- Assign a field to store the text length.

In [9]:
import chromadb
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [ ]:
# read api_key from file
with open('../api_keys.txt', 'r') as file:
    api_key = file.read()


# Set loaded api_key as OPENAI_API_KEY environmental variable
import os
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:

llm = ChatOpenAI(model='gpt-3.5-turbo')

In [13]:
# Define prompt
prompt = ChatPromptTemplate.from_template("Explain {topic} in 20 words.")


# Convert LLM output to a dictionary so `.assign()` can process it
convert_to_dict_runnable = RunnableLambda(lambda x: {"content": x.content})


# Create pipeline with `.assign()`
pipeline = (prompt | llm | convert_to_dict_runnable).assign(
    length=lambda inputs: len(inputs["content"])
)

# Invoke the pipeline
output = pipeline.invoke({"topic": "quantum computing"})
print(output)

{'content': 'Quantum computing uses quantum bits (qubits) to perform complex calculations exponentially faster than classical computers, revolutionizing data processing.', 'length': 156}


### **👀Example 3: Combining `.assign()` with Parallel Execution**
**Scenario:**
- Run two different LLM chains in parallel.
- Assign a new field that counts total characters.

In [ ]:
# Correctly define prompts using from_messages()
angry_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an angry person. Respond to: {input}")
])



happy_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a happy person. Respond to: {input}")
])


# Define a parallel runnable
runnable = RunnableParallel(
                        angry_response = angry_prompt | llm | RunnableLambda(lambda x: x.content),
                        happy_response = happy_prompt | llm  | RunnableLambda(lambda x: x.content)
        ) | RunnablePassthrough.assign( # assigning a new field to the output dictionary
                                total_chars=lambda inputs: len(inputs['angry_response'] + inputs['happy_response'])
                                )
        



# Invoke the pipeline
output = runnable.invoke({"input": "hello"})
print(output)

{'angry_response': "What do you want? Can't you see I'm busy?", 'happy_response': "Hello! I hope you're having a wonderful day.", 'total_chars': 85}


### ☕️ Same Logic, Different Syntax


```python 
runnable = {
    'angry_response':  angry_prompt | llm | RunnableLambda(lambda x: x.content),
    'happy_response':  appy_prompt | llm  | RunnableLambda(lambda x: x.content),
} 
``` 



### **Attention!**
You can achieve the same result using `RunnableLambda`

In [44]:
from langchain.schema.runnable import RunnableLambda


# Correctly define prompts using from_messages()
angry_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an angry person. Respond to: {input}")
])

happy_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a happy person. Respond to: {input}")
])



# Define a RunnableLambda that calculates total characters
calculate_total_chars = RunnableLambda(lambda inputs: {
    **inputs,  # Keep the original dictionary keys and values ({angry_response: ... ,happy_response: ... })
    'total_chars': len(inputs['angry_response'] + inputs['happy_response'])  # Compute total character count
})


# Define a parallel runnable
runnable = RunnableParallel(
    angry_response=angry_prompt | llm | RunnableLambda(lambda x: x.content),
    happy_response=happy_prompt | llm  | RunnableLambda(lambda x: x.content)
) | calculate_total_chars  # Apply the RunnableLambda



# Invoke the pipeline
output = runnable.invoke({"input": "hello"})
print(output)


{'angry_response': "What do you want? Don't waste my time with meaningless greetings.", 'happy_response': 'Hello! I hope you are doing well today. How can I make your day even brighter?', 'total_chars': 143}


In [45]:
runnable.get_graph().print_ascii()

   +----------------------------------------------+    
   | Parallel<angry_response,happy_response>Input |    
   +----------------------------------------------+    
                  ***             ***                  
                **                   **                
              **                       **              
+--------------------+          +--------------------+ 
| ChatPromptTemplate |          | ChatPromptTemplate | 
+--------------------+          +--------------------+ 
           *                               *           
           *                               *           
           *                               *           
    +------------+                  +------------+     
    | ChatOpenAI |                  | ChatOpenAI |     
    +------------+                  +------------+     
           *                               *           
           *                               *           
           *                               *    

### Compare `assign()` in `RunnableParallel` and `RunnableSequence`

In [25]:
def return_hundred(input):
    return 100

In [37]:
chain = RunnablePassthrough().assign(from_assign=RunnableLambda(return_hundred))

result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

{'input': 'hello', 'input2': 'goodbye', 'from_assign': 100}


In [38]:
chain = RunnableParallel({"from_parallel": RunnablePassthrough()}).assign(from_assign=RunnableLambda(return_hundred))

result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

{'from_parallel': {'input': 'hello', 'input2': 'goodbye'}, 'from_assign': 100}


## **👾 Good to know**
LangChain automatically treats a dictionary as a `RunnableParallel` only when it's used inside a pipeline (e.g., `|` pipe). But if you're working with it standalone (like trying to call `.assign()` on it), you must explicitly wrap it.

`chain = {"from_parallel": RunnablePassthrough()}.assign(from_assign=RunnableLambda(return_hundred))` throws an error.

**Below is the right way to implicit create a `RunnableParallel` via Dictionary**

In [42]:
# Single key dictionary interpreted as parallel runnable
chain = {
    "from_parallel": RunnablePassthrough()
} | RunnablePassthrough().assign(from_assign=RunnableLambda(return_hundred))



result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

{'from_parallel': {'input': 'hello', 'input2': 'goodbye'}, 'from_assign': 100}


## 🔧 What is `.bind()` in LangChain?

## 📌 Definition
In LangChain LCEL (LangChain Expression Language), **`.bind()`** is used to **pre-fill fixed values** into a Runnable's expected input.  
It’s like “locking in” a variable so you don’t need to provide it every time you call `.invoke()`.

---

## ✅ What Does `.bind()` Do?

- It returns a new `Runnable` with the **specified input(s) already filled in**.
- The bound inputs are automatically merged with the ones you provide at runtime.
- Great for **customizing reusable chains**, **fixing configuration values**, or **simplifying input**.

---

## ✅ **`.bind()` — Fix Certain Inputs**

In [ ]:
def render_message(name, product):
    return f"Hello {name}, thank you for buying {product}!"


langchain_thank_you_note = RunnableLambda(render_message).bind(product="LangChain Pro")



print(langchain_thank_you_note.invoke("Sarah"))  # Output: "Hello Sarah, thank you for buying LangChain Pro!"

Hello Sarah, thank you for buying LangChain Pro!


In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a suepr angry and rude customer."),
    ("user", "respond to this {greeting}")
    ]
)


def print_runnable(data: dict) -> dict:
    print(data)
    return data


# Bind the system message
chain = langchain_thank_you_note  | print_runnable | chat_prompt | print_runnable | llm


chain.invoke("Sarah").content

Hello Sarah, thank you for buying LangChain Pro!
messages=[SystemMessage(content="You're a suepr angry and rude customer."), HumanMessage(content='respond to this Hello Sarah, thank you for buying LangChain Pro!')]


"About time you thanked me, but that's not enough! I demand top-notch customer service and a discount on my next purchase for all the trouble I've had with your product. Get your act together and start treating your customers with the respect they deserve!"

##  **✅`.bind(stop=[, , ,])`**

In LangChain LCEL, `.bind(stop=[[, , ,])` is used to **configure generation behavior** for an LLM by specifying **stop sequences** — strings that tell the model when to stop generating tokens.

---

## 📌 What Is `stop`?

The `stop` parameter is a list of strings.  
When the model sees **any of these strings** in the output, it will **immediately stop generating** further text.

---

In [69]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)

model = ChatOpenAI(temperature=0)

runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION: 
Subtract 7 from both sides:
x^3 = 5

Take the cube root of both sides:
x = ∛5


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "Write out the following equation using algebraic symbols then solve it. Use the format\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)

model = ChatOpenAI(temperature=0)

runnable = (
    {"equation_statement": RunnablePassthrough()} 
    | prompt 
    | model.bind(stop=['SOLUTION']) 
    | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12



# 🔧 `.bind()` vs `.assign()` in LangChain LCEL

| Feature       | `.bind()`                                                  | `.assign()`                                                |
|---------------|------------------------------------------------------------|------------------------------------------------------------|
| 🧠 Purpose     | Pre-fill or lock in input values                           | Add new fields to the output dictionary                    |
| 🎯 Scope       | Affects **inputs before** the Runnable is executed        | Affects **outputs after** the Runnable is executed         |
| 🧪 Behavior    | Replaces or fills in part of the input                     | Computes extra fields based on the result of a Runnable    |
| 🧱 Input Type  | Typically used before chains or prompts                    | Typically used on output dictionaries                     |
| 📥 Example     | `chain.bind(topic="AI")`                                   | `chain.assign(length=lambda x: len(x["text"]))`           |
| 📤 Output      | Same as original chain, but with fewer required inputs     | Returns original output **plus** new key(s)                |
| 🔁 Reusability | Great for creating reusable, pre-configured chains         | Great for enriching outputs for further steps              |

---

## 🧠 In Short:

- Use **`.bind()`** to **simplify your inputs**.
- Use **`.assign()`** to **add data to your outputs**.